In [3]:
from scipy.sparse.construct import rand
import torch, random, os, multiprocessing
import numpy as np, pandas as pd, nibabel as nib 
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchio as tio
# multiprocess cpu 
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
from utils.S1_utils import clip_gradient
num_workers = multiprocessing.cpu_count()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True
random.seed(1234)
torch.manual_seed(1234)

In [2]:
csv_path = './NIHSS_Continuous_Variable_222patient.csv'
table_ =  pd.read_csv(csv_path, index_col=False)
# print(table_.columns)
table_label = table_.drop(['ID','rnn_sum(out-in)'] ,axis=1)
# print(table_.columns)                           
print("table_label.columns.values", len(table_label.columns.values))
# table_label.to_csv('./NIHSS_fiter_222patient.csv',index=False)
# print(table_label)

Index(['ID', 'A/P_01', 'N/L_01', 'age', 'gender',
       'onset-to-image(有acute-發病日期)', 'onset-to-ward(病房日期-發病日期)', 'NIHSS: 1a',
       'NIHSS: 1b', 'NIHSS: 1c', 'NIHSS: 2', 'NiHSS: 3', 'NIHSS: 4',
       'NIHSS: 5a', 'NIHSS: 5b', 'NIHSS: 6a', 'NIHSS:6b', 'NIHSS:7', 'NIHSS:8',
       'NIHSS:9', 'NIHSS:10', 'NIHSS:11', 'NIHSS  total', 'out_sum',
       'rnn_sum(out-in)'],
      dtype='object')
table_label.columns.values 23
     A/P_01  N/L_01  age  gender  onset-to-image(有acute-發病日期)  \
0         1       1   86       1                            6   
1         1       0   89       0                            0   
2         0       0   78       1                            2   
3         0       1   91       1                            3   
4         0       0   61       1                            4   
..      ...     ...  ...     ...                          ...   
217       1       1   59       0                            1   
218       0       0   80       1                      

In [3]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(table_label)

y_nor = scaler.transform(table_label)
# print(y_nor)
new_table_nor = pd.DataFrame(y_nor, columns=table_label.columns)

X_table = new_table_nor.drop(['out_sum'] ,axis=1)
y_table = new_table_nor['out_sum']

In [4]:
from torch.utils.data import TensorDataset, DataLoader
X_train, X_test, y_train, y_test = train_test_split(np.array(X_table), np.array(y_table), test_size=0.25, random_state=123) #seed = 42, 123
print("train", y_train.shape, "test", y_test.shape)
training_set = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
validation_set = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))


train (166,) test (56,)


In [5]:
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.layer = torch.nn.Linear(22, 1)

   def forward(self, x):
       x = self.layer(x)      
       return x

net = Net().to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
loss_func = torch.nn.MSELoss()
print(net)

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch):
   model.train()
   stream = tqdm(train_loader)
   for i, (value, count) in enumerate(stream, start=1):
      # print(nihss)
      output = model(value.to(device))
      # print(output)
      loss = criterion(output, count.to(device))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      stream.set_description(f"Epoch: {epoch}. Train. loss: {loss}")

def validate(valid_loader, model, criterion, epoch):
   global best_vloss, best_vacc
   model.eval()
   stream_v = tqdm(valid_loader)
   with torch.no_grad():
      for i, (value, count) in enumerate(stream_v, start=1):
         # print(nihss)
         output = model(value.to(device))
         # print(output)
         loss = criterion(output, count.to(device))
         stream_v.set_description(f"Epoch: {epoch}. Valid. loss: {loss}")
# X
def  train_valid_process_main(model, training_set, validation_set, batch_size):
   global best_vloss, best_vacc
   best_vloss = np.inf
   best_vacc = 0.00
   # Subject Dataloader Building
   train_loader = DataLoader(training_set, batch_size=8, shuffle=True, num_workers=10)
   valid_loader = DataLoader(validation_set, batch_size=1, shuffle=False, num_workers=4)

   for epoch in range(1, 200 + 1):
      train(train_loader, model, loss_func, optimizer, epoch)
      validate(valid_loader, model, loss_func, epoch)
   return model

In [ ]:
logs  = train_valid_process_main(net, training_set, validation_set, 8)